In [1]:
from mzpricer import option_greeks, option_iv, option_price, StockPrice, TimeDuration, OptionType
import pandas as pd 
import polars as pl
cols = [
    "Ticker",            # e.g., AAPL
    "ValuationTime",     # ISO8601, e.g., 2025-09-11T15:30:00
    "Spot",              # Underlying spot price S
    "Type",              # 'C' for call, 'P' for put
    "Strike",            # K
    "Expiry",            # ISO date or datetime for option expiration
    "Rate",              # risk-free cont. comp. rate r (annualized, decimal)
    "DividendYield",     # continuous dividend yield q (annualized, decimal)
    "Vol30d",            # annualized vol sigma (decimal, e.g., 0.25)
    "ContractMultiplier",# usually 100
    "Bid",               # market bid
    "Ask",               # market ask
    "Mid"                # market mid you will compare to fair
]

In [2]:
df = pl.read_csv("../UnderlyingOptionsEODCalcs_2023-08-25_cgi_or_historical.csv")#.filter(pl.col("underlying_symbol") == "TSLA").head(1000)

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the ticker symbol and date range
ticker_symbol = "TSLA"
start_date = "2023-06-25"
end_date = "2023-08-25"

def get_price_series(df, ticker_symbol, prefer_adj=True):
    if isinstance(df.columns, pd.MultiIndex):
        cols = df.columns
        if prefer_adj and ('Adj Close', ticker_symbol) in cols:
            return df[('Adj Close', ticker_symbol)].rename('Adj Close')
        if ('Close', ticker_symbol) in cols:
            return df[('Close', ticker_symbol)].rename('Close')
    else:
        if prefer_adj and 'Adj Close' in df.columns:
            return df['Adj Close']
        if 'Close' in df.columns:
            return df['Close']
    raise KeyError("Could not find a price column ('Adj Close' or 'Close').")


# Download the data
tsla_data = yf.download(ticker_symbol, start=start_date, end=end_date)
price = get_price_series(tsla_data, 'TSLA', prefer_adj=True)
# Ensure returns are based on adjusted close
tsla_data['Return'] = price.pct_change()

# Compute 30-day rolling volatility (annualized)
# As of 2023-08-23 close
window = 30
trading_days = 252  # typical number of trading days in a year
tsla_data['Volatility_30d'] = (
    tsla_data['Return'].rolling(window).std() * np.sqrt(trading_days)
)

# Get the volatility on 2023-08-23
vol_20230823 = tsla_data.loc["2023-08-23", "Volatility_30d"]
vol_20230823

/var/folders/8_/y69hhps94q5cvrnr_2hlmnq80000gp/T/ipykernel_93282/2082510388.py:26: FutureWarning: YF.download() has changed argument auto_adjust default to True
  tsla_data = yf.download(ticker_symbol, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


Ticker
    0.487469
Name: 2023-08-23 00:00:00, dtype: float64

In [4]:

input_data = (df.filter(pl.col("underlying_symbol") == "TSLA")
 .with_columns(
    pl.datetime(
        year=2023,
        month=8,
        day=23,
        hour=16,
        minute=0,
        second=0,
        time_zone="America/New_York",
 ).alias("ValuationTime"),
 ((pl.col("underlying_ask_1545") + pl.col("underlying_bid_1545"))/2).alias("underlying_mid_1545"),
 (
     pl.col("expiration").str.to_datetime(format="%Y-%m-%d", time_zone="America/New_York") + pl.duration(hours=14)
 ).dt.to_string("iso"),
 pl.lit(100).alias('ContractMultiplier'),
 pl.lit(.05).alias("temp_borrow_rate"),
 pl.lit(0.0).alias("temp_div"),
 pl.lit(vol_20230823[0]).alias("temp_30dvol"),
 ((pl.col('ask_1545') + pl.col('bid_1545'))/2).alias('mid_1545')
 )

 .rename({
     "underlying_symbol": "Ticker", 
    "underlying_mid_1545": "Spot",             
    "option_type": "Type",             
    "strike": "Strike",           
    "expiration": "Expiry",           
    "temp_borrow_rate": "Rate",             
    "temp_div": "DividendYield",    
    "temp_30dvol": "Vol30d",           
    "ContractMultiplier": "ContractMultiplier",
    "bid_1545": "Bid",              
    "ask_1545": "Ask",              
    "mid_1545": "Mid"               
 })
 .select(cols)
)

def _yearfrac_act365(t0: pd.Timestamp, t1: pd.Timestamp) -> float:
    return max((t1 - t0).total_seconds(), 0) / (365.0 * 24 * 3600)


/var/folders/8_/y69hhps94q5cvrnr_2hlmnq80000gp/T/ipykernel_93282/2891844631.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pl.lit(vol_20230823[0]).alias("temp_30dvol"),


In [5]:

vt = pd.to_datetime(
    input_data["ValuationTime"].to_list(),
    utc=False,
    errors="coerce"
)
ex = pd.to_datetime(
    input_data["Expiry"].to_list(),
    utc=False,
    errors="coerce"
)

T = np.array([_yearfrac_act365(t0, t1) if (pd.notna(t0) and pd.notna(t1)) else np.nan
                for t0, t1 in zip(vt, ex)])

T = [TimeDuration(t, 365) for t in T]

/var/folders/8_/y69hhps94q5cvrnr_2hlmnq80000gp/T/ipykernel_93282/1559548676.py:6: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  ex = pd.to_datetime(


In [6]:
S = input_data.select(pl.col('Spot')).to_numpy()   # Current Stock Price
# s_pricer = StockPrice(spot_price=S, dividend_amount=0.0, time_to_dividend_days=0.0, rate=0.0) 
# s_prime = s_pricer.s_prime()  # Adjusted stock price considering dividends
K = input_data.select(pl.col('Strike')).to_numpy()   # Strike Price
T = T     # Time to Expiration (1 year)
R = [.0525] * len(K)    # Risk-Free Rate (5%)
SIGMA = input_data.select(pl.col('Vol30d')).to_numpy() # Volatility (20%)
opt_type = [OptionType.Call if r == 'C' else OptionType.Put for r in input_data.select(pl.col('Type')).to_numpy()]

market_price = input_data.select(pl.col('Mid')).to_numpy()

In [7]:
price, error = option_price(S, K, T, R, SIGMA, opt_type, 200)


/var/folders/8_/y69hhps94q5cvrnr_2hlmnq80000gp/T/ipykernel_93282/2688830136.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  price, error = option_price(S, K, T, R, SIGMA, opt_type, 200)


In [8]:
from mzpricer import option_greeks
# greeks = option_greeks(S, K, T, R, SIGMA, opt_type, 100)


In [9]:
# market_vol, error = option_iv(market_price, S, K, T, R, SIGMA, opt_type)

In [11]:
output = input_data.with_columns(
    pl.Series(price).alias("fair"),
    # pl.Series(market_vol).alias("vol")
)

In [15]:
output.filter((pl.col("fair").is_between(pl.col("Bid"), pl.col("Ask")) ))

Ticker,ValuationTime,Spot,Type,Strike,Expiry,Rate,DividendYield,Vol30d,ContractMultiplier,Bid,Ask,Mid,fair
str,"datetime[μs, America/New_York]",f64,str,f64,str,f64,f64,f64,i32,f64,f64,f64,f64
"""TSLA""",2023-08-23 16:00:00 EDT,238.33,"""C""",20.0,"""2023-08-25 14:00:00.000000-04:…",0.05,0.0,0.487469,100,218.2,218.35,218.275,218.330015
"""TSLA""",2023-08-23 16:00:00 EDT,238.33,"""P""",20.0,"""2023-08-25 14:00:00.000000-04:…",0.05,0.0,0.487469,100,0.0,0.01,0.005,0.0
"""TSLA""",2023-08-23 16:00:00 EDT,238.33,"""C""",30.0,"""2023-08-25 14:00:00.000000-04:…",0.05,0.0,0.487469,100,208.2,208.35,208.275,208.330023
"""TSLA""",2023-08-23 16:00:00 EDT,238.33,"""P""",30.0,"""2023-08-25 14:00:00.000000-04:…",0.05,0.0,0.487469,100,0.0,0.01,0.005,0.0
"""TSLA""",2023-08-23 16:00:00 EDT,238.33,"""C""",40.0,"""2023-08-25 14:00:00.000000-04:…",0.05,0.0,0.487469,100,198.2,198.35,198.275,198.33003
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""TSLA""",2023-08-23 16:00:00 EDT,238.33,"""P""",520.0,"""2025-12-19 14:00:00.000000-05:…",0.05,0.0,0.487469,100,278.35,286.4,282.375,281.67
"""TSLA""",2023-08-23 16:00:00 EDT,238.33,"""P""",530.0,"""2025-12-19 14:00:00.000000-05:…",0.05,0.0,0.487469,100,287.95,296.1,292.025,291.67
"""TSLA""",2023-08-23 16:00:00 EDT,238.33,"""P""",540.0,"""2025-12-19 14:00:00.000000-05:…",0.05,0.0,0.487469,100,297.9,306.0,301.95,301.67


In [ ]:
output.group_by("Expiry").agg([
    pl.count().alias("count"),
    pl.col("vol").filter(pl.col("vol").is_between(0.2,4.9)).count().alias("vol_mismatch")
]).with_columns([
    (pl.col("vol_mismatch") / pl.col("count") * 100).alias("vol_mismatch_pct")
]).sort("vol_mismatch_pct", descending=False)

/var/folders/8_/y69hhps94q5cvrnr_2hlmnq80000gp/T/ipykernel_63086/2239681168.py:2: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("count"),


Expiry,count,vol_mismatch,vol_mismatch_pct
str,u32,u32,f64
"""2025-09-19 14:00:00.000000-04:…",190,134,70.526316
"""2025-06-20 14:00:00.000000-04:…",164,126,76.829268
"""2025-12-19 14:00:00.000000-05:…",154,120,77.922078
"""2024-09-20 14:00:00.000000-04:…",212,172,81.132075
"""2025-01-17 14:00:00.000000-05:…",162,136,83.950617
…,…,…,…
"""2023-09-29 14:00:00.000000-04:…",182,170,93.406593
"""2023-09-22 14:00:00.000000-04:…",182,172,94.505495
"""2023-09-15 14:00:00.000000-04:…",416,394,94.711538


In [ ]:
t = output.filter((pl.col("Vol30d") != pl.col("vol")))

In [ ]:
t

Ticker,ValuationTime,Spot,Type,Strike,Expiry,Rate,DividendYield,Vol30d,ContractMultiplier,Bid,Ask,Mid,fair,vol
str,"datetime[μs, America/New_York]",f64,str,f64,str,f64,f64,f64,i32,f64,f64,f64,f64,f64
"""TSLA""",2023-08-23 16:00:00 EDT,238.33,"""C""",237.5,"""2023-08-25 14:00:00.000000-04:…",0.05,0.0,0.487469,100,0.85,0.9,0.875,0.835202,0.799907
"""TSLA""",2023-08-23 16:00:00 EDT,238.33,"""P""",237.5,"""2023-08-25 14:00:00.000000-04:…",0.05,0.0,0.487469,100,0.05,0.07,0.06,0.005023,0.876947
"""TSLA""",2023-08-23 16:00:00 EDT,238.33,"""C""",240.0,"""2023-08-25 14:00:00.000000-04:…",0.05,0.0,0.487469,100,0.02,0.03,0.025,0.000008,1.157798
"""TSLA""",2023-08-23 16:00:00 EDT,238.33,"""P""",240.0,"""2023-08-25 14:00:00.000000-04:…",0.05,0.0,0.487469,100,1.67,1.74,1.705,1.67,1.248507
"""TSLA""",2023-08-23 16:00:00 EDT,238.33,"""C""",232.5,"""2023-09-01 14:00:00.000000-04:…",0.05,0.0,0.487469,100,9.25,9.3,9.275,5.830817,5.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""TSLA""",2023-08-23 16:00:00 EDT,238.33,"""P""",300.0,"""2025-12-19 14:00:00.000000-05:…",0.05,0.0,0.487469,100,96.05,97.4,96.725,61.67,5.0
"""TSLA""",2023-08-23 16:00:00 EDT,238.33,"""C""",310.0,"""2025-12-19 14:00:00.000000-05:…",0.05,0.0,0.487469,100,58.6,59.65,59.125,1.0253e-11,5.0
"""TSLA""",2023-08-23 16:00:00 EDT,238.33,"""P""",310.0,"""2025-12-19 14:00:00.000000-05:…",0.05,0.0,0.487469,100,102.85,104.2,103.525,71.67,5.0


In [ ]:
vt = pd.to_datetime(
    ['2023-08-23 16:00:00.000000-04'],
    utc=False,
    errors="coerce"
)
ex = pd.to_datetime(
    ["2023-08-23 16:00:00.000000-04"],
    utc=False,
    errors="coerce"
)

T = np.array([_yearfrac_act365(t0, t1) if (pd.notna(t0) and pd.notna(t1)) else np.nan
                for t0, t1 in zip(vt, ex)])

T = [TimeDuration(t, 365) for t in T]

In [ ]:
S = [238.33]   # Current Stock Price
# s_pricer = StockPrice(spot_price=S, dividend_amount=0.0, time_to_dividend_days=0.0, rate=0.0) 
# s_prime = s_pricer.s_prime()  # Adjusted stock price considering dividends
K = [237.5]  # Strike Price
T = T    # Time to Expiration (1 year)
R = [.0525]     # Risk-Free Rate (5%)
SIGMA = [0.487469] # Volatility (20%)
opt_type = [OptionType.Call]

market_price = [0.875]

In [ ]:
print(S,K, T, R, SIGMA, opt_type, market_price)

[238.33] [237.5] [<builtins.TimeDuration object at 0x1610ff7e0>] [0.0525] [0.487469] [<OptionType.Call: 0>] [0.875]


In [ ]:
market_vol, error = option_iv(market_price, S, K, T, R, [.88], opt_type)

In [ ]:
market_vol

[0.88]

In [ ]:
T
